In [529]:
# example of a wgan for generating handwritten digits
from numpy import expand_dims
from numpy import mean
from numpy import ones
from numpy.random import randn
from numpy.random import randint
from keras.datasets.mnist import load_data
from keras import backend
from tensorflow.keras.optimizers import RMSprop
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import BatchNormalization
from keras.initializers import RandomNormal
from keras.constraints import Constraint
from matplotlib import pyplot
import pandas as pd
import numpy as np
import plotly.express as px 

In [530]:
df1 = pd.read_csv ("../Data/train.csv")
df = np.array(df1)[:,1:]
df = df[:, [0, 1]]
df, df.shape

(array([[0.01143886, 0.00269057],
        [0.00063166, 0.0072774 ],
        [0.01782796, 0.02821015],
        ...,
        [0.00768299, 0.00195759],
        [0.00339559, 0.00127959],
        [0.0045912 , 0.00667492]]),
 (745, 2))

In [531]:
# calculate wasserstein loss
def wasserstein_loss(y_true, y_pred):
	return backend.mean(y_true * y_pred)

In [532]:
def define_generator(G_in, dim=128, out_dim=2, lr=1e-3):
    x = Dense(dim)(G_in)
    x = Dense(dim*2)(x)
    x = Dense(dim*4)(x)
    G_out = Dense(out_dim)(x)
    G = Model(G_in, G_out)
    opt = RMSprop(lr=lr)
    G.compile(loss=wasserstein_loss, optimizer=opt)
    return G,G_out

G_in = Input(shape=[2])
G, G_out = define_generator(G_in)
G.summary()

Model: "model_73"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_85 (InputLayer)        [(None, 2)]               0         
_________________________________________________________________
dense_299 (Dense)            (None, 128)               384       
_________________________________________________________________
dense_300 (Dense)            (None, 256)               33024     
_________________________________________________________________
dense_301 (Dense)            (None, 512)               131584    
_________________________________________________________________
dense_302 (Dense)            (None, 2)                 1026      
Total params: 166,018
Trainable params: 166,018
Non-trainable params: 0
_________________________________________________________________


C:\Users\BEN ALLAL\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\optimizer_v2\optimizer_v2.py:355: UserWarning:

The `lr` argument is deprecated, use `learning_rate` instead.



In [533]:
# define the standalone critic model
def define_critic(D_in,dim=128,out_dim=1,lr=1e-3):
	const = ClipConstraint(0.01)
	x = Dense(dim * 4)(D_in)
	x = Dropout(0.1)(x)
	x = Dense(dim * 2)(x)
	x = Dropout(0.1)(x)
	x = Dense(dim)(x)
	D_out = Dense(out_dim)(x)
	D = Model(D_in, D_out)
	opt = RMSprop(lr=lr)
	D.compile(loss=wasserstein_loss, optimizer=opt)
	return D,D_out

D_in = Input(shape=[2])
D, D_out = define_critic(D_in)
D.summary()

Model: "model_74"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_86 (InputLayer)        [(None, 2)]               0         
_________________________________________________________________
dense_303 (Dense)            (None, 512)               1536      
_________________________________________________________________
dropout_54 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_304 (Dense)            (None, 256)               131328    
_________________________________________________________________
dropout_55 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_305 (Dense)            (None, 128)               32896     
_________________________________________________________________
dense_306 (Dense)            (None, 1)                 129

In [534]:
def set_trainability(model, trainable=False):
    model.trainable = trainable
    for layer in model.layers:
        layer.trainable = trainable
        
def define_gan(GAN_in, G, D):
    set_trainability(D, False)
    x = G(GAN_in)
    GAN_out = D(x)
    GAN = Model(GAN_in, GAN_out)
    GAN.compile(loss=wasserstein_loss, optimizer=G.optimizer)
    return GAN, GAN_out

GAN_in = Input([2])
GAN, GAN_out = define_gan(GAN_in, G, D)
GAN.summary()

Model: "model_75"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_87 (InputLayer)        [(None, 2)]               0         
_________________________________________________________________
model_73 (Functional)        (None, 2)                 166018    
_________________________________________________________________
model_74 (Functional)        (None, 1)                 165889    
Total params: 331,907
Trainable params: 166,018
Non-trainable params: 165,889
_________________________________________________________________


In [535]:
def train(GAN, G, D, epochs, batch_size=1,latent_dim=4,verbose=False):
    valid = -np.ones((batch_size, 1))
    fake = np.ones((batch_size, 1))
    d_loss = []
    g_loss = []
    clip_value = 0.01
    n_critic = 5
    e_range = range(epochs)
    f = open('loss_logs.csv','w')
    f.write('Epoch,Discriminator Loss,Generator Loss\n')

    if verbose:
        e_range = tqdm(e_range)
    for epoch in e_range:
        for _ in range(n_critic):
            idx = np.random.randint(0, df.shape[0], batch_size)
            batch_data = df[idx]
                
            # Sample noise as generator input
            noise = np.random.normal(-1, 1, (batch_size, latent_dim))

            # Generate a batch of new images
            gen_data = G.predict(noise)
            
            d_loss_real = D.train_on_batch(batch_data, valid)
            d_loss_fake = D.train_on_batch(gen_data, fake)
            d_loss = 0.5 * np.add(d_loss_fake, d_loss_real)
            
            for l in D.layers:
                weights = l.get_weights()
                weights = [np.clip(w, -clip_value, clip_value) for w in weights]
                l.set_weights(weights)
            
        g_loss = GAN.train_on_batch(noise, valid)

        # Plot the progress
        print ("%d [D loss: %f] [G loss: %f]" % (epoch, d_loss, g_loss))
        f.write("%d,%f,%f\n"%(epoch,d_loss,g_loss))

In [536]:
train(GAN, G, D, epochs=100, batch_size=1,latent_dim=2,verbose=False)

0 [D loss: 0.000992] [G loss: -0.002269]
1 [D loss: 0.004183] [G loss: -0.003881]
2 [D loss: 0.011038] [G loss: -0.010478]
3 [D loss: -0.106620] [G loss: 0.352194]
4 [D loss: -0.170291] [G loss: 0.469816]
5 [D loss: 0.030136] [G loss: -0.041948]
6 [D loss: -0.484465] [G loss: 1.089784]
7 [D loss: 0.019768] [G loss: -0.032022]
8 [D loss: 0.017463] [G loss: -0.026331]
9 [D loss: 0.011532] [G loss: -0.015268]
10 [D loss: 0.001929] [G loss: -0.003554]
11 [D loss: 0.001196] [G loss: -0.002204]
12 [D loss: 0.000424] [G loss: -0.001016]
13 [D loss: 0.000662] [G loss: -0.000880]
14 [D loss: -0.019533] [G loss: 0.071593]
15 [D loss: 0.002255] [G loss: -0.006270]
16 [D loss: 0.000822] [G loss: -0.002663]
17 [D loss: 0.000734] [G loss: -0.000241]
18 [D loss: -0.000028] [G loss: 0.000396]
19 [D loss: 0.000830] [G loss: -0.001605]
20 [D loss: 0.000817] [G loss: -0.001202]
21 [D loss: 0.000684] [G loss: -0.001398]
22 [D loss: 0.000490] [G loss: -0.001276]
23 [D loss: 0.000498] [G loss: -0.001293]
24

In [537]:
data = pd.read_csv('../Notebooks/loss_logs.csv')
fig = px.line(data, x='Epoch', y=data.columns[1:3])
fig.show()